### H2O

https://www.cienciadedatos.net/documentos/py04_machine_learning_con_h2o_y_python

In [1]:
!pip3 install h2o

You should consider upgrading via the 'z:\data_science\software\python\python_3.6.4\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

import h2o
from h2o.automl import H2OAutoML

In [3]:
train = pd.read_csv('diamonds_train.csv')
test = pd.read_csv('diamonds_test.csv')

In [4]:
# hay que instalar JAVA versión < 14
# https://www.oracle.com/java/technologies/javase-jdk11-downloads.html

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.231-b11, mixed mode)
  Starting server from Z:\Data_Science\Software\Python\Python_3.6.4\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\gvazquez\AppData\Local\Temp\tmphlb9pm_2
  JVM stdout: C:\Users\gvazquez\AppData\Local\Temp\tmphlb9pm_2\h2o_gvazquez_started_from_python.out
  JVM stderr: C:\Users\gvazquez\AppData\Local\Temp\tmphlb9pm_2\h2o_gvazquez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,"7 days, 7 hours and 4 minutes"
H2O_cluster_name:,H2O_from_python_gvazquez_r9vhb3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"accepting new members, healthy"


In [6]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
columnas = [a for a in h2train.columns if a != "price"][1:]
x = columnas
y = "price"
print(columnas)

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']


In [8]:
automl = H2OAutoML(max_models=50, seed=42, max_runtime_secs=300, sort_metric='RMSE')
automl.train(x=x, y=y, training_frame=h2train)
# en Windows no está disponible XGBoost :'(

AutoML progress: |
20:28:54.967: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [17]:
print('[INFO] Models leader board:')
leader_board = automl.leaderboard
leader_board.head()

[INFO] Models leader board:


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210208_202854,527.63,278393,278393,264.85,0.0924574
StackedEnsemble_BestOfFamily_AutoML_20210208_202854,533.292,284400,284400,266.606,0.0927642
GBM_4_AutoML_20210208_202854,533.602,284731,284731,268.38,0.0924719
GBM_3_AutoML_20210208_202854,533.943,285095,285095,272.376,0.0982032
GBM_grid__1_AutoML_20210208_202854_model_5,534.292,285468,285468,272.373,nan
GBM_2_AutoML_20210208_202854,534.413,285598,285598,275.044,nan
GBM_grid__1_AutoML_20210208_202854_model_2,537.035,288407,288407,270.383,0.0934097
GBM_grid__1_AutoML_20210208_202854_model_6,544.811,296819,296819,270.494,0.095192
GBM_grid__1_AutoML_20210208_202854_model_1,545.413,297476,297476,281.774,0.105801
GBM_grid__1_AutoML_20210208_202854_model_7,554.468,307435,307435,286.886,0.10671


In [18]:
predictions = automl.leader.predict(h2test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [19]:
test.rename(columns={"Unnamed: 0" : "id"}, inplace = True)

In [20]:
test["price"] = predictions.as_data_frame()
columnasentrega = ["id","price"]
entrega = test["price"]
test = test[columnasentrega]

In [21]:
test.head()

,id,price
0,0,412.603792
1,1,1560.843046
2,2,9584.753949
3,3,509.110144
4,4,10088.244120


In [22]:
test['price'].astype("int")

0          412
1         1560
2         9584
3          509
4        10088
         ...  
13444     4957
13445      446
13446    14928
13447    10839
13448      712
Name: price, Length: 13449, dtype: int32

In [23]:
test.to_csv('to_submit_int.csv', index=False)

In [26]:
testing = pd.read_csv('to_submit_int.csv')
testing

,id,price
0,0,412.603792
1,1,1560.843046
2,2,9584.753949
3,3,509.110144
4,4,10088.244120
...,...,...
13444,13444,4957.286328
13445,13445,446.418057
13446,13446,14928.467755
13447,13447,10839.254557


In [27]:
h2o.cluster().shutdown()

H2O session _sid_9a68 was not closed properly.


**Evaluación con los datos reales**

In [28]:
true = pd.read_csv('evaluation_price.csv')

y_pred = testing['price']
y_true = true['price']
np.sqrt(mean_squared_error(y_true, y_pred))

530.4868276331717